In [7]:
import re
import pandas as pd
from smart_open import smart_open

rx_dict = {
  'map' : re.compile(r'map\s+(?P<topic>\d+)\s+(?P<value>.*)\n'),
  'prec': re.compile(r'Rprec\s+(?P<topic>\d+)\s+(?P<value>.*)\n'),
  'p10' : re.compile(r'P_10\s+(?P<topic>\d+)\s+(?P<value>.*)\n')
}

def _parse_line(line):
  for key, rx in rx_dict.items():
    match = rx.search(line)
    if match:
      return key, match
  return None, None

def parse_file(filepath):
  data = []
  with smart_open(filepath, 'r') as file_object:
    record = {}
    for line in file_object:
      key, match = _parse_line(line) 
      if match is None:
        continue
      topic = int(match.group('topic'))
      if bool(record.get('topic')) and record.get('topic') != topic:
        data.append(record.copy()) 
      record['topic'] = topic
      value = float(match.group('value'))
      if key == 'map':
        record['AP'] = value
      elif key == 'prec':
        record['prec'] = value
      elif key == 'p10':
        record['p10'] = value
    if bool(record): data.append(record.copy())
    data = pd.DataFrame(data, columns=['topic', 'AP', 'prec', 'p10'])
    data.set_index('topic', inplace=True)
    return data

In [13]:
a=parse_file('C:\\Users\Matteo\Desktop\IR\BM25 PS & NSL\BM25.txt');
print(list(a['AP']))

[0.0255, 0.0, 0.0346, 0.0044, 0.0118, 0.0005, 0.0, 0.0452, 0.007, 0.073, 0.0054, 0.0173, 0.0106, 0.0275, 0.0293, 0.034, 0.0072, 0.0237, 0.0012, 0.0001, 0.0029, 0.0149, 0.0025, 0.0934, 0.0098, 0.0005, 0.0296, 0.0, 0.0076, 0.0036, 0.0001, 0.0077, 0.0039, 0.0001, 0.0257, 0.0053, 0.0512, 0.033, 0.0017, 0.0296, 0.0, 0.0034, 0.0028, 0.0015, 0.0068, 0.0133, 0.0148, 0.0008, 0.0077, 0.0816]
